In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

NUE = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2"

#if ISRUN3:
#    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
#    
#else:
#    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    
UPROOTNUE = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "shr_tkfit_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg",  "n_tracks_contained", 
    "NeutrinoEnergy2",
    "nu_flashmatch_score",
    "isVtxInFiducial","truthFiducial","nproton",
    "elec_e","proton_e","muon_e",
    "topological_score",
    "trk_energy_muon",
    "npion",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist0","trkshrhitdist1","trkshrhitdist2", # distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    "pi0_gammadot","pi0_mass_Y","pi0_energy2_Y", # pi0 tagger variables
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", 
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
]

nue = UPROOTNUE.pandas.df(variables, flatten=False)

In [ ]:
def EQE(df,elecangle,elecenergy,ke):
    mp = 0.998
    mn = 0.939
    Eb = 0.00987 # see DocDB 26949
    theta = df[elecangle]
    El = df[elecenergy]
    if (ke == True):
        El += 0.105
    ml = 0.105
    num = mp**2 - (mn-Eb)**2 - ml**2 + 2*(mn-Eb)*El
    den = 2*(mn-Eb-El + np.sqrt(El**2-ml**2) * np.cos(theta))
    return num/den

In [ ]:
# define some energy-related variables
# calorimetric energy
nue["reco_e_calo"] = nue["NeutrinoEnergy2"]/1000. + 0.105
# muon from range + all other tracks as protons from range
nue["reco_e_range"] = nue["trk_energy_muon"] + (nue["trk_energy_tot"] - nue["trk_energy"]) + 0.105
# CCQE energy
nue["reco_e_qe"] = EQE(nue,'trk_theta','trk_energy_muon',1)
nue["reco_e_qe_truth"] = EQE(nue,'trk_theta','muon_e',0)

In [ ]:
# energy resolution for calorimetric energy
nue['erescalo'] = (nue['reco_e_calo']-nue['nu_e'])/nue['nu_e']
# energy resolution for calorimetric energy vs. visible energy
nue['erescalovis'] = (nue['reco_e_calo']-nue['true_e_visible']-0.105)/nue['true_e_visible']
# energy resolution for calorimetric energy
nue['eresrange'] = (nue['reco_e_range']-nue['nu_e'])/nue['nu_e']
# energy resolution for calorimetric energy vs. visible energy
nue['eresrangevis'] = (nue['reco_e_range']-nue['true_e_visible']-0.105)/nue['true_e_visible']
# energy resolution for muon
nue['muonres'] = (nue['trk_energy_muon']-(nue['muon_e']-0.105))/(nue['muon_e']-0.105)
nue['recores'] = (nue['reco_e_calo']-nue['reco_e_range'])/nue['reco_e_calo']

nue['eresqe'] = (nue['reco_e_qe']-nue['nu_e'])/nue['nu_e']
nue['eresqetruth'] = (nue['reco_e_qe_truth']-nue['nu_e'])/nue['nu_e']

In [ ]:
trk_llr_pid_v = UPROOTNUE.array('trk_llr_pid_score_v')
trk_id = UPROOTNUE.array('trk_id')-1 # I think we need this -1 to get the right result
trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
nue['trkpid'] = trk_llr_pid_v_sel
nue['subcluster'] = nue['shrsubclusters0'] + nue['shrsubclusters1'] + nue['shrsubclusters2']
nue['trkfit'] = nue['shr_tkfit_npointsvalid'] / nue['shr_tkfit_npoints']

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
QUERY += ' and topological_score > 0.06'
QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
QUERY += ' and trkpid > 0.2'

QUERY += ' and truthFiducial == 0'

print (QUERY)

In [ ]:
nue_SEL = nue.query(QUERY)

In [ ]:
nueRES = nue_SEL.query('interaction == 1')
nueQE  = nue_SEL.query('interaction == 0')
nueDIS = nue_SEL.query('interaction == 2')
nueMEC = nue_SEL.query('interaction == 10')

nue0pi = nue_SEL.query('npion == 0 and npi0 == 0')
nueNpi = nue_SEL.query('npion != 0 or npi0 != 0')

In [ ]:
fig = plt.figure(figsize=(6,6))
BINS2D = (np.linspace(0.15,1.5,50),np.linspace(0.15,1.5,50))
plt.hist2d(nue_SEL['nu_e'].values,nue_SEL['reco_e_range'].values,bins=BINS2D)
plt.grid()
plt.xlabel('True Neutrino Energy [GeV]')
plt.ylabel('Reconstructed Range-Based Energy [GeV]')
plt.title(r'truth non-contained $\nu_{\mu}$ events')
plt.show()

In [ ]:
ERANGE_V = []

ERANGE_V = [0.1,0.2,0.3,0.4,0.5,0.6,0.8,1.0,1.5,2.0]


for n in range(len(ERANGE_V)-1):
    
    EMIN = ERANGE_V[n]
    EMAX = ERANGE_V[n+1]
    
    stacked_v = []
    
    VAR = 'eresrange'

    nueQE_tmp = nueQE.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))
    nueRES_tmp = nueRES.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))
    nueMEC_tmp = nueMEC.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))
    nueDIS_tmp = nueDIS.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))
    
    nue0pi_tmp = nue0pi.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))
    nueNpi_tmp = nueNpi.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))
    
    stacked_v = [nueDIS_tmp[VAR],nueRES_tmp[VAR],\
                 nueMEC_tmp[VAR],nueQE_tmp[VAR]]

    label_v = ['DIS','RES','MEC','QE']
    
    fig = plt.figure(figsize=(6,6))
    BINS = np.linspace(-1,1,30)
    plt.hist(stacked_v,bins=BINS,histtype='stepfilled',lw=2,label=label_v,stacked=True,alpha=0.5)
    plt.grid()
    plt.legend(loc="best",framealpha=1,fontsize=16)
    plt.xlabel('[reco - true] / true energy',fontsize=16)
    plt.title(r'true $\nu_{\mu}$ energy [%.01f, %.01f] GeV'%(EMIN,EMAX),fontsize=16)
    plt.show()
    
    stacked_v = [nueNpi_tmp[VAR],nue0pi_tmp[VAR]]

    label_v = ['N pion','0 pion']
    
    fig = plt.figure(figsize=(6,6))
    BINS = np.linspace(-1,1,30)
    plt.hist(stacked_v,bins=BINS,histtype='stepfilled',lw=2,label=label_v,stacked=True,alpha=0.5)
    plt.grid()
    plt.legend(loc="best",framealpha=1,fontsize=16)
    plt.xlabel('[calo - range] / calo energy',fontsize=16)
    plt.title(r'true $\nu_{\mu}$ energy [%.01f, %.01f] GeV'%(EMIN,EMAX),fontsize=16)
    plt.show()

In [ ]:
ERANGE_V = []

ERANGE_V = [0.1,0.2,0.3,0.4,0.5,0.6,0.8,1.0,1.5,2.0]


for n in range(len(ERANGE_V)-1):
    
    EMIN = ERANGE_V[n]
    EMAX = ERANGE_V[n+1]

    nue_SEL_tmp = nue_SEL.query('nu_e > %f and nu_e < %f'%(EMIN,EMAX))

    fig = plt.figure(figsize=(6,6))
    BINS = np.linspace(-1,1,30)
    #plt.hist(nue_SEL_tmp['eresclo'].values,bins=BINS,histtype='step',lw=2,label='Calorimetric Energy')
    #plt.hist(nue_SEL_tmp['eresvis'].values,bins=BINS,histtype='step',lw=2,label='Calorimetric Energy [hermeticity]')
    plt.hist(nue_SEL_tmp['eresqe'].values,bins=BINS,histtype='step',lw=2,label='QE Energy')
    plt.hist(nue_SEL_tmp['eresqetruth'].values,bins=BINS,histtype='step',lw=2,label='QE Energy - Truth')
    plt.grid()
    plt.legend(loc=3,framealpha=1,fontsize=16)
    plt.xlabel('[reco - true] / true energy',fontsize=16)
    plt.title(r'true $\nu_e$ energy [%.01f, %.01f] GeV'%(EMIN,EMAX),fontsize=16)
    plt.show()

In [ ]:
ERANGE_V = []

ERANGE_V = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.5,0.6,0.7,0.8,0.9,1.0]


for n in range(len(ERANGE_V)-1):
    
    EMIN = ERANGE_V[n]
    EMAX = ERANGE_V[n+1]

    nue_SEL_tmp = nue_SEL.query('muon_e > %.02f and muon_e < %.02f'%(EMIN+0.105,EMAX+0.105))

    fig = plt.figure(figsize=(4,4))
    BINS = np.linspace(-0.5,0.5,100)
    plt.hist(nue_SEL_tmp['muonres'].values,bins=BINS,histtype='step',lw=2)#,label='Calorimetric Energy')
    plt.grid()
    plt.legend(loc=1)
    plt.xlabel('[reco - true] / true energy',fontsize=16)
    plt.title('true muon KE [%.02f, %.02f] GeV'%(EMIN,EMAX),fontsize=16)
    plt.show()

In [ ]:
def Eff(df,var,query,acceptance,bin_edges,absval=False):
    #print acceptance
    bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
    bins = []
    bin_eff = []
    bin_err = []
    for i in range(len(bin_centers)):
        binmin = bin_edges[i]
        binmax = bin_edges[i+1]
        bincut = '%s > %f and %s < %f'%(var,binmin,var,binmax)
        if (absval == True):
            bincut = '(%s > %f and %s < %f) or (%s > -%f and %s < -%f)'%(var,binmin,var,binmax,var,binmax,var,binmin)
        if (acceptance != ''): bincut += ' and %s'%acceptance
        #print bincut
        df_tmp =  df.query(bincut) # cut on bin range for desired var.
        df_sub = df_tmp.query(query) # apply constrain 
        if (df_tmp.shape[0] == 0): continue
        eff = df_sub.shape[0] / float( df_tmp.shape[0] )
        err = np.sqrt( eff*(1-eff)/df_tmp.shape[0] )
        bin_eff.append( eff )
        bin_err.append( err )
        bins.append(bin_centers[i])
        #print 'eff = %.02f @ bin = %.02f'%(eff,bin_centers[i])
    return np.array(bins),np.array(bin_eff),np.array(bin_err)

In [ ]:
# nue pre-selection
BOXCUTS = 'nslice == 1'
BOXCUTS += ' and selected == 1'
BOXCUTS += ' and n_tracks_contained > 0'
BOXCUTS += ' and shr_energy_tot_cali > 0.07'
BOXCUTS += ' and n_tracks_contained > 0'
# Loose box cuts
BOXCUTS += ' and n_showers_contained == 1'
BOXCUTS += ' and hits_ratio > 0.5'
BOXCUTS += ' and tksh_distance < 6.0'
BOXCUTS += ' and shr_tkfit_2cm_dedx_Y < 4.0'
BOXCUTS += ' and tksh_angle > -0.9'
BOXCUTS += ' and trkpid < 0.1'
BOXCUTS += ' and shr_score < 0.30'
BOXCUTS += ' and CosmicIP > 20.'
# Box cuts
BOXCUTS += ' and tksh_distance < 3.5'
BOXCUTS += ' and (shr_tkfit_gap10_dedx_Y > 0 and shr_tkfit_gap10_dedx_Y < 4.5 and shr_tkfit_gap10_dedx_U < 4.5 and shr_tkfit_gap10_dedx_V < 4.5)'
BOXCUTS += ' and (shr_tkfit_2cm_dedx_Y > 0 and shr_tkfit_2cm_dedx_Y < 4.0 and shr_tkfit_2cm_dedx_U < 4.0 and shr_tkfit_2cm_dedx_V < 4.0)'
BOXCUTS += ' and tksh_angle > -0.9 and tksh_angle < 0.7'
BOXCUTS += ' and shrmoliereavg > 2 and shrmoliereavg < 9'
BOXCUTS += ' and trkpid < -0.02'
BOXCUTS += ' and n_showers_contained == 1'
BOXCUTS += ' and shr_score < 0.275'
BOXCUTS += ' and hits_ratio > 0.60'
BOXCUTS += ' and CosmicIP > 20.'
BOXCUTS += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
BOXCUTS += ' and (trkfit < 0.45 or subcluster > 6)'

In [ ]:
PRESEL = 'nslice == 1'
PRESEL += ' and selected == 1'
PRESEL += ' and n_tracks_contained > 0'
PRESEL += ' and shr_energy_tot_cali > 0.07'
PRESEL += ' and trkpid < -0.02'

In [ ]:
# muon selection
MUQUERY = 'nslice == 1'
#if ISRUN3: MUQUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#MUQUERY += ' and trk_len > 20'
MUQUERY += ' and topological_score > 0.06'
MUQUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
MUQUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
MUQUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
#MUQUERY += ' and trkpid > 0.8'

In [ ]:
fig = plt.figure(figsize=(8,6))
B = np.linspace(0.1,1.5,15)

#ACCEPTANCE = 'isVtxInFiducial == 1 and truthFiducial == 1 and nproton > 0'
ACCEPTANCE = 'isVtxInFiducial == 1 and ccnc == 0'
#QUERY = SELQUERY
VAR = 'nu_e' #'_pi0_e'

centers,vals,errs = Eff(nue,VAR,'nslice==1',ACCEPTANCE,B)
plt.errorbar(centers,vals,yerr=errs,xerr=0.05,fmt='o'\
             ,color='b',label=r'Neutrino SlideID on $\nu_{\mu}$ CC')

centers,vals,errs = Eff(nue,VAR,MUQUERY,ACCEPTANCE,B)
plt.errorbar(centers,vals,yerr=errs,xerr=0.05,fmt='o',\
             color='m',label=r'$\nu_{\mu}$ selection on $\nu_{\mu}$ CC')

ACCEPTANCE = 'isVtxInFiducial == 1 and ccnc == 1'
#QUERY = SELQUERY
VAR = 'nu_e' #'_pi0_e'

#centers,vals,errs = Eff(nue,VAR,'nslice==1',ACCEPTANCE,B)
#plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='r',label='Neutrino SlideID')

#centers,vals,errs = Eff(nue,VAR,PRESEL,ACCEPTANCE,B)
#plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='r',label='1eNp reconstruction [3.5% purity]')

#centers,vals,errs = Eff(nue,VAR,BOXCUTS,ACCEPTANCE,B)
#plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='g',label='LEE box-cut selection [91% purity]')

plt.xlabel(r'Neutrino Energy [GeV]',fontsize=16)
plt.ylabel(r'Efficiency',fontsize=16)
plt.grid()
plt.ylim([0,1.3])
plt.legend(loc=1,fontsize=16,framealpha=1.0)
#plt.yscale('log')
#plt.title(r'v08_00_00_26 Run1 Samples')
#plt.legend(loc=7,fontsize=14)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
B = np.linspace(0.0,1.0,50)

ACCEPTANCE = 'isVtxInFiducial == 1 and truthFiducial == 1 and nproton > 0'
#QUERY = SELQUERY
VAR = 'elec_e' #'_pi0_e'

centers,vals,errs = Eff(nue,VAR,'nslice==1',ACCEPTANCE,B)
plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='b',label='Neutrino SlideID [0.4% purity]')

centers,vals,errs = Eff(nue,VAR,PRESEL,ACCEPTANCE,B)
plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='r',label='1eNp reconstruction [3.5% purity]')

centers,vals,errs = Eff(nue,VAR,BOXCUTS,ACCEPTANCE,B)
plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='g',label='LEE box-cut selection [91% purity]')

plt.xlabel(r'Electron Energy [GeV]',fontsize=16)
plt.ylabel(r'Efficiency',fontsize=16)
plt.grid()
plt.ylim([0,1.3])
plt.legend(loc=9,fontsize=16,framealpha=1.0)
#plt.yscale('log')
#plt.title(r'v08_00_00_26 Run1 Samples')
#plt.legend(loc=7,fontsize=14)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
B = np.linspace(0.938,1.4,50)

ACCEPTANCE = 'isVtxInFiducial == 1 and truthFiducial == 1 and nproton > 0'
#QUERY = SELQUERY
VAR = 'proton_e' #'_pi0_e'


PRESEL = 'nslice == 1'
PRESEL += ' and selected == 1'
PRESEL += ' and n_tracks_contained > 0'
PRESEL += ' and shr_energy_tot_cali > 0.07'

centers,vals,errs = Eff(nue,VAR,PRESEL,ACCEPTANCE,B)
centers -= 0.938
plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='r',label='1eNp reconstruction')

PRESEL = 'nslice == 1'
PRESEL += ' and selected == 1'
PRESEL += ' and n_tracks_contained > 0'
PRESEL += ' and shr_energy_tot_cali > 0.07'
PRESEL += ' and trkpid < -0.02'

centers,vals,errs = Eff(nue,VAR,PRESEL,ACCEPTANCE,B)
centers -= 0.938
plt.errorbar(centers,vals,yerr=errs,fmt='o-',color='g',label='1eNp reconstruction [w/ proton PID]')

plt.xlabel(r'Proton Kinetic Energy [GeV]',fontsize=16)
plt.ylabel(r'Efficiency',fontsize=16)
plt.grid()
plt.ylim([0,1.0])
plt.legend(loc=9,fontsize=16,framealpha=1.0)
#plt.yscale('log')
#plt.title(r'v08_00_00_26 Run1 Samples')
#plt.legend(loc=7,fontsize=14)
plt.show()